# Install dan Download Category Encoder


In [ ]:
pip install category_encoders

     |████████████████████████████████| 81kB 2.1MB/s 


# Install Ipython Autotime

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 221 µs (started: 2021-01-09 02:51:59 +00:00)


# Import Libraries

In [ ]:
import keras
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import svm
from sklearn.model_selection import KFold
from category_encoders.target_encoder import TargetEncoder

from sklearn import tree
import matplotlib.pyplot as plt

from category_encoders.target_encoder import TargetEncoder

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


time: 3.14 s (started: 2021-01-09 02:51:59 +00:00)


# Load Data

Read data dan drop kolom yang tidak dibutuhkan

In [ ]:
df = pd.read_csv("winemag-data_first150k.csv")
df.drop(columns=['description', 'Unnamed: 0', 'region_2'], inplace=True)
df

,country,designation,points,price,province,region_1,variety,winery
0,US,Martha's Vineyard,96,235.0,California,Napa Valley,Cabernet Sauvignon,Heitz
1,Spain,Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sauvignon Blanc,Macauley
3,US,Reserve,96,65.0,Oregon,Willamette Valley,Pinot Noir,Ponzi
4,France,La Brûlade,95,66.0,Provence,Bandol,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...
150925,Italy,NaN,91,20.0,Southern Italy,Fiano di Avellino,White Blend,Feudi di San Gregorio
150926,France,Cuvée Prestige,91,27.0,Champagne,Champagne,Champagne Blend,H.Germain
150927,Italy,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,White Blend,Terredora
150928,France,Grand Brut Rosé,90,52.0,Champagne,Champagne,Champagne Blend,Gosset


time: 786 ms (started: 2021-01-09 02:52:02 +00:00)


# Membersihkan Data

Assign df_disc dengan data yang tidak memiliki nilai null

In [ ]:
df_disc = df.copy()
for col in df_disc.columns:
    df_disc = df_disc[df_disc[col].notnull()]

df_disc

,country,designation,points,price,province,region_1,variety,winery
0,US,Martha's Vineyard,96,235.0,California,Napa Valley,Cabernet Sauvignon,Heitz
1,Spain,Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sauvignon Blanc,Macauley
3,US,Reserve,96,65.0,Oregon,Willamette Valley,Pinot Noir,Ponzi
4,France,La Brûlade,95,66.0,Provence,Bandol,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...
150923,France,Demi-Sec,91,30.0,Champagne,Champagne,Champagne Blend,Jacquart
150924,France,Diamant Bleu,91,70.0,Champagne,Champagne,Champagne Blend,Heidsieck & Co Monopole
150926,France,Cuvée Prestige,91,27.0,Champagne,Champagne,Champagne Blend,H.Germain
150927,Italy,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,White Blend,Terredora


time: 149 ms (started: 2021-01-09 02:52:03 +00:00)


Assign df_disc dengan data yang pada kolom "winery" memiliki nilai yang duplicate

In [ ]:
df_disc = df_disc[df_disc.duplicated(subset=["winery"], keep=False)]
print(df_disc)

       country  ...                   winery
0           US  ...                    Heitz
1        Spain  ...  Bodega Carmen Rodríguez
2           US  ...                 Macauley
3           US  ...                    Ponzi
4       France  ...     Domaine de la Bégude
...        ...  ...                      ...
150923  France  ...                 Jacquart
150924  France  ...  Heidsieck & Co Monopole
150926  France  ...                H.Germain
150927   Italy  ...                Terredora
150928  France  ...                   Gosset

[75408 rows x 8 columns]
time: 43.3 ms (started: 2021-01-09 02:52:03 +00:00)


Assign df_disc dengan data yang pada kolom "designation" memiliki nilai yang duplicate

In [ ]:
df_disc = df_disc[df_disc.duplicated(subset=["designation"], keep=False)]
print(df_disc)

       country  ...                   winery
0           US  ...                    Heitz
3           US  ...                    Ponzi
5        Spain  ...                Numanthia
6        Spain  ...                 Maurodos
9           US  ...                Blue Farm
...        ...  ...                      ...
150923  France  ...                 Jacquart
150924  France  ...  Heidsieck & Co Monopole
150926  France  ...                H.Germain
150927   Italy  ...                Terredora
150928  France  ...                   Gosset

[66668 rows x 8 columns]
time: 44.6 ms (started: 2021-01-09 02:52:03 +00:00)


Assign df_disc dengan data yang pada kolom "region_1" memiliki nilai yang duplicate

In [ ]:
df_disc = df_disc[df_disc.duplicated(subset=["region_1"], keep=False)]
print(df_disc)

       country  ...                   winery
0           US  ...                    Heitz
3           US  ...                    Ponzi
5        Spain  ...                Numanthia
6        Spain  ...                 Maurodos
9           US  ...                Blue Farm
...        ...  ...                      ...
150923  France  ...                 Jacquart
150924  France  ...  Heidsieck & Co Monopole
150926  France  ...                H.Germain
150927   Italy  ...                Terredora
150928  France  ...                   Gosset

[66605 rows x 8 columns]
time: 37.2 ms (started: 2021-01-09 02:52:03 +00:00)


Assign df_disc dengan data yang pada kolom "variety" memiliki nilai yang duplicate

In [ ]:
df_disc = df_disc[df_disc.duplicated(subset=["variety"], keep=False)]
print(df_disc)

       country  ...                   winery
0           US  ...                    Heitz
3           US  ...                    Ponzi
5        Spain  ...                Numanthia
6        Spain  ...                 Maurodos
9           US  ...                Blue Farm
...        ...  ...                      ...
150923  France  ...                 Jacquart
150924  France  ...  Heidsieck & Co Monopole
150926  France  ...                H.Germain
150927   Italy  ...                Terredora
150928  France  ...                   Gosset

[66571 rows x 8 columns]
time: 37.5 ms (started: 2021-01-09 02:52:03 +00:00)


Mendiskritkan target menjadi kontinu menggunakan qcut dari pandas dengan parameter pd.qcut(kolom, jumlah_kelas, label) dan replace kolom target (price) dengan angka

In [ ]:
df_disc['points'] = pd.qcut(df_disc['points'], 2, labels=['low', 'high'])
df_disc['points'].replace({'low': 0, 'high': 1}, inplace=True)
df_disc

,country,designation,points,price,province,region_1,variety,winery
0,US,Martha's Vineyard,1,235.0,California,Napa Valley,Cabernet Sauvignon,Heitz
3,US,Reserve,1,65.0,Oregon,Willamette Valley,Pinot Noir,Ponzi
5,Spain,Numanthia,1,73.0,Northern Spain,Toro,Tinta de Toro,Numanthia
6,Spain,San Román,1,65.0,Northern Spain,Toro,Tinta de Toro,Maurodos
9,US,Gap's Crown Vineyard,1,60.0,California,Sonoma Coast,Pinot Noir,Blue Farm
...,...,...,...,...,...,...,...,...
150923,France,Demi-Sec,1,30.0,Champagne,Champagne,Champagne Blend,Jacquart
150924,France,Diamant Bleu,1,70.0,Champagne,Champagne,Champagne Blend,Heidsieck & Co Monopole
150926,France,Cuvée Prestige,1,27.0,Champagne,Champagne,Champagne Blend,H.Germain
150927,Italy,Terre di Dora,1,20.0,Southern Italy,Fiano di Avellino,White Blend,Terredora


time: 87.4 ms (started: 2021-01-09 02:52:03 +00:00)


# Encoding

Menggunakan Target Encoder yang meng-encode berdasarkan target. Value yang diberikan antara 0 sampai 1

In [ ]:
te = TargetEncoder()

y = df_disc['points'].values
X = df_disc.drop(columns=['points']).values
te.fit(X, y)
X = te.transform(X)

X.head()

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,0,1,2,3,4,5,6
0,0.462723,0.851715,0.999498,0.452261,0.557334,0.531222,0.851715
1,0.462723,0.372807,0.783105,0.557117,0.516393,0.598954,0.795918
2,0.270993,0.999498,0.787879,0.296432,0.439614,0.542169,1.000000
3,0.270993,0.973851,0.783105,0.296432,0.439614,0.542169,0.899944
4,0.462723,0.781250,0.759768,0.452261,0.671525,0.598954,0.973851


time: 535 ms (started: 2021-01-09 02:52:03 +00:00)


# Training

## ANN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=1)

time: 13.7 ms (started: 2021-01-09 02:52:04 +00:00)


In [ ]:
model=keras.models.Sequential([
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),

    keras.layers.Dense(units=1, activation="sigmoid"),
],name="Initial_model",)
model.summary()

Model: "Initial_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               4096      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 168,449
Trainable params: 168,449
Non-trainable params: 0
_________________________________________________________________
time: 267 ms (started: 2021-01-09 02:52:04 +00:00)


In [ ]:
learning_rate = 0.005
optimizer = keras.optimizers.Adam(lr=learning_rate)

time: 2.18 ms (started: 2021-01-09 02:52:04 +00:00)


In [ ]:
model.compile(optimizer=optimizer,
            loss='binary_crossentropy', metrics=['accuracy'])

time: 27.5 ms (started: 2021-01-09 02:52:04 +00:00)


In [ ]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=1000,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

time: 2.77 ms (started: 2021-01-09 02:52:04 +00:00)


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=500, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1, callbacks=[early_stopping_monitor])

Epoch 1/500
20/20 [==============================] - 2s 67ms/step - loss: 0.4752 - accuracy: 0.7742 - val_loss: 0.3165 - val_accuracy: 0.8653
Epoch 2/500
20/20 [==============================] - 1s 45ms/step - loss: 0.2963 - accuracy: 0.8724 - val_loss: 0.2781 - val_accuracy: 0.8771
Epoch 3/500
20/20 [==============================] - 1s 44ms/step - loss: 0.2836 - accuracy: 0.8747 - val_loss: 0.2726 - val_accuracy: 0.8789
Epoch 4/500
20/20 [==============================] - 1s 50ms/step - loss: 0.2722 - accuracy: 0.8758 - val_loss: 0.2701 - val_accuracy: 0.8777
Epoch 5/500
20/20 [==============================] - 1s 44ms/step - loss: 0.2713 - accuracy: 0.8749 - val_loss: 0.2688 - val_accuracy: 0.8778
Epoch 6/500
20/20 [==============================] - 1s 43ms/step - loss: 0.2695 - accuracy: 0.8758 - val_loss: 0.2605 - val_accuracy: 0.8803
Epoch 7/500
20/20 [==============================] - 1s 44ms/step - loss: 0.2614 - accuracy: 0.8803 - val_loss: 0.2582 - val_accuracy: 0.8791
Epoch 

In [ ]:
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)

time: 2.31 s (started: 2021-01-09 02:59:36 +00:00)


In [ ]:
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.963, Test: 0.869
time: 1.75 ms (started: 2021-01-09 02:59:39 +00:00)


## Decision Tree

In [ ]:
param_dt = {
    'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],
    'max_depth': [3, 5, 7],
    'criterion': ['gini', 'entropy']
}

time: 2.42 ms (started: 2021-01-09 02:59:39 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_dt) + 2)
print(row_format.format('t_size', 'max_depth', 'criterion', 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_dt['test_size']:
  for j in param_dt['max_depth']:
    for k in param_dt['criterion']:
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
      dt = DecisionTreeClassifier(random_state=42, max_depth=j, criterion=k)
      dt.fit(X_train, y_train)

      # print(y_test)
      pred = dt.predict(X_test)
      acc = metrics.accuracy_score(y_test, pred)
      f1 = metrics.f1_score(y_test, pred)
      if(acc > acc_bfr):
        acc_bfr = acc
        best_param_dt = {'t_size': i, 'max_depth': j, 'criterion': k, 'accuracy': acc, 'f1': f1}
        best_dt = dt
      print(row_format.format(str(i), str(j), str(k), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_dt)     

    t_size max_depth criterion   akurasi        f1
       0.9         3      gini     0.871     0.863
       0.9         3   entropy     0.870     0.863
       0.9         5      gini     0.875     0.859
       0.9         5   entropy     0.877     0.866
       0.9         7      gini     0.868     0.850
       0.9         7   entropy     0.875     0.862
       0.7         3      gini     0.871     0.864
       0.7         3   entropy     0.869     0.862
       0.7         5      gini     0.876     0.863
       0.7         5   entropy     0.877     0.865
       0.7         7      gini     0.876     0.863
       0.7         7   entropy     0.878     0.867
       0.5         3      gini     0.871     0.864
       0.5         3   entropy     0.865     0.860
       0.5         5      gini     0.877     0.866
       0.5         5   entropy     0.876     0.865
       0.5         7      gini     0.874     0.857
       0.5         7   entropy     0.876     0.859
       0.3         3      gini 

## SVM

In [ ]:
param_svm = {
  'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],  
  'C'    : [1, 10, 100, 0.1],
  'gamma': [0.1, 0.01, 0.001, 1.0],
  'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

time: 2.66 ms (started: 2021-01-09 02:59:41 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_svm) + 2)
print(row_format.format(*param_svm, 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_svm['test_size']:
  for j in param_svm['C']:
    for k in param_svm['gamma']:
      for l in param_svm['kernel']:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
        svc = SVC(C=j, gamma=k, kernel=l)
        svc.fit(X_train, y_train)

        # print(y_test)
        pred = svc.predict(X_test)
        acc = metrics.accuracy_score(y_test, pred)
        f1 = metrics.f1_score(y_test, pred)
        if(acc > acc_bfr):
          acc_bfr = acc
          best_param_svm = {'t_size': i, 'C': j, 'gamma': k, 'kernel': l, 'accuracy': acc, 'f1': f1}
        print(row_format.format(str(i), str(j), str(k), str(l), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_svm)     

 test_size         C     gamma    kernel   akurasi        f1
       0.9         1       0.1    linear     0.878     0.863
       0.9         1       0.1      poly     0.865     0.840
       0.9         1       0.1       rbf     0.877     0.862
       0.9         1       0.1   sigmoid     0.878     0.863
       0.9         1      0.01    linear     0.878     0.863
       0.9         1      0.01      poly     0.551     0.000
       0.9         1      0.01       rbf     0.877     0.862
       0.9         1      0.01   sigmoid     0.877     0.861
       0.9         1     0.001    linear     0.878     0.863
       0.9         1     0.001      poly     0.551     0.000
       0.9         1     0.001       rbf     0.874     0.857
       0.9         1     0.001   sigmoid     0.873     0.854
       0.9         1       1.0    linear     0.878     0.863
       0.9         1       1.0      poly     0.877     0.857
       0.9         1       1.0       rbf     0.878     0.863
       0.9         1    

## KNN

In [ ]:
param_knn = {
  'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],  
  'n_neighbor': [3, 5],
  'weights': ['uniform', 'distance'],
  'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
}

time: 2.23 ms (started: 2021-01-09 07:19:30 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_knn) + 2)
print(row_format.format(*param_knn, 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_knn['test_size']:
  for j in param_knn['n_neighbor']:
    for k in param_knn['weights']:
      for l in param_knn['algorithm']:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
        knn = KNeighborsClassifier(n_neighbors=j, weights=k, algorithm=l)
        knn.fit(X_train, y_train)

        # print(y_test)
        pred = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, pred)
        f1 = metrics.f1_score(y_test, pred)
        if(acc > acc_bfr):
          acc_bfr = acc
          best_param_knn = {'t_size': i, 'n_beighbors': j, 'weights': k, 'algorithm': l, 'accuracy': acc, 'f1': f1}
        print(row_format.format(str(i), str(j), str(k), str(l), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_knn)     

 test_sizen_neighbor   weights algorithm   akurasi        f1
       0.9         3   uniform      auto     0.862     0.845
       0.9         3   uniform ball_tree     0.862     0.846
       0.9         3   uniform   kd_tree     0.862     0.845
       0.9         3   uniform     brute     0.862     0.845
       0.9         3  distance      auto     0.864     0.847
       0.9         3  distance ball_tree     0.864     0.847
       0.9         3  distance   kd_tree     0.864     0.847
       0.9         3  distance     brute     0.864     0.847
       0.9         5   uniform      auto     0.869     0.853
       0.9         5   uniform ball_tree     0.869     0.853
       0.9         5   uniform   kd_tree     0.869     0.853
       0.9         5   uniform     brute     0.869     0.853
       0.9         5  distance      auto     0.869     0.853
       0.9         5  distance ball_tree     0.870     0.853
       0.9         5  distance   kd_tree     0.869     0.853
       0.9         5  di